In [ ]:
#!/usr/bin/env python
# coding: utf-8

'''
import SQL
'''

# In[ ]:
# ** import package **
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import traceback
from tqdm import tqdm
from datetime import timedelta
from _utils.customlogger import customlogger as CL

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

In [ ]:
# In[ ]:
# ** loading config **
with open('./../{}'.format("config.json")) as file:
    cfg = json.load(file)

In [ ]:
# In[ ]:
# ** loading info **
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
curr_file_name = os.path.splitext(os.path.basename(os.path.abspath('')))[0]
data_dir = pathlib.Path('{}/data/'.format(parent_dir))
pathlib.Path.mkdir(data_dir, mode=0o777, parents=True, exist_ok=True)

In [ ]:
# In[ ]:
# **create Logger**
log = CL("custom_logger")
pathlib.Path.mkdir(pathlib.Path('{}/_log/'.format(parent_dir)), mode=0o777, parents=True, exist_ok=True)
log = log.create_logger(file_name="../_log/{}.log".format(curr_file_name), mode="a", level="DEBUG")  
log.debug('start {}'.format(curr_file_name))

In [ ]:
# In[ ]:
# ** connection DataBase **
if (cfg["dbms"]=="postgresql"):
    db_cfg = cfg["postgresql"]
    import psycopg2 as pg
    conn = pg.connect(host=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], dbname=db_cfg['@database']) 
    log.debug("postgresql connect")
    
elif (cfg["dbms"]=="mssql"):
    db_cfg = cfg["mssql"]
    import pymssql
    conn= pymssql.connect(server=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], database=db_cfg['@database'], as_dict=False)
    log.debug("mssql connect")
    
elif (cfg["dbms"]=="mysql"):
    db_cfg = cfg["mysql"]
    import pymysql
    conn= pymysql.connect(host=db_cfg['@server'], user=db_cfg['@user'], password=db_cfg['@password'], port=db_cfg['@port'], db=db_cfg['@database'], charset='utf8', autocommit=True)
    log.debug("mysql connect")
    
else:
    log.warning("set config.json - sql - dbms : mssql or postgresql")

In [ ]:
apachepsvar_df = pd.read_sql("SELECT * FROM eicu.APACHEAPSVAR", con=conn)
patient_df = pd.read_sql("SELECT * FROM eicu.PATIENT", con=conn)
microlab_df = pd.read_sql("SELECT * FROM eicu.MICROLAB", con=conn)
lab_df = pd.read_sql("SELECT * FROM eicu.LAB", con=conn)
medication_df = pd.read_sql("SELECT * FROM eicu.MEDICATION", con=conn)
admissiondrug_df = pd.read_sql("SELECT * FROM eicu.ADMISSIONDRUG", con=conn)

In [ ]:
apachepsvar_df.to_feather('apachepsvar_df.feather')
patient_df.to_feather('patient_df.feather')
microlab_df.to_feather('microlab_df.feather')
lab_df.to_feather('lab_df.feather')
medication_df.to_feather('medication_df.feather')
admissiondrug_df.to_feather('admissiondrug_df.feather')

In [ ]:
apachepsvar_df = pd.read_feather('apachepsvar_df.feather')
patient_df = pd.read_feather('patient_df.feather')
microlab_df = pd.read_feather('microlab_df.feather')
lab_df = pd.read_feather('lab_df.feather')
medication_df = pd.read_feather('medication_df.feather')
admissiondrug_df = pd.read_feather('admissiondrug_df.feather')

In [ ]:
patient_apc_df = pd.merge(patient_df, apachepsvar_df, left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_med_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], medication_df[['patientunitstayid', 'drugname']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_lab_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], lab_df[['patientunitstayid', 'labname', 'labresult']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
patient_adm_df = pd.merge(patient_df[['patientunitstayid', 'unittype']], admissiondrug_df[['patientunitstayid', 'drugname']], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [ ]:
patient_med_df.rename(columns={'drugname':'concept_name'}, inplace=True)
patient_med_df['concept_value'] = 1
patient_med_df['concept_table'] = 'med'
patient_lab_df.rename(columns={'labname':'concept_name','labresult':'concept_value'}, inplace=True)
patient_lab_df['concept_table'] = 'lab'
patient_adm_df.rename(columns={'drugname':'concept_name'}, inplace=True)
patient_adm_df['concept_value'] = 1
patient_adm_df['concept_table'] = 'adm'

In [ ]:
concat_df = pd.concat([patient_med_df, patient_lab_df, patient_adm_df], axis=0, ignore_index=True)

In [ ]:
apache_cols_df = pd.DataFrame(apachepsvar_df.columns, columns=['concept_name'])
apache_cols_df['concept_table'] = 'apc'
patient_cols_df = pd.DataFrame(patient_df.columns, columns=['concept_name'])
patient_cols_df['concept_table'] = 'pat'
remains_cols_df = concat_df[['concept_name', 'concept_table']]
concat_cols_df = pd.concat([patient_cols_df, apache_cols_df, remains_cols_df], axis=0).drop_duplicates().reset_index(drop=True)
concat_cols_df.to_feather('{}/all_cols_df.feather'.format(data_dir))


In [ ]:
def filter_with_missing_rate(df, threshold):
    def filter_with_missing_rate_concept(x, nTotalPatients, threshold):
        nConceptPatients = len(x.patientunitstayid.unique())
        fEpsilon = 1.0e-08 # devide by zero
        fMissingRate = nConceptPatients / (nTotalPatients + fEpsilon)
        # if (fMissingRate < threshold) :
        #     return pd.DataFrame(columns=x.columns)
        # print("{}, {}, {}, {:.2}", list(x.concept_name)[0], nConceptPatients, nTotalPatients, fMissingRate)
        temp_df = pd.DataFrame()
        var_temp = {}
        var_temp['concept_name'] = list(x.concept_name)[0]
        var_temp['concept_table'] = list(x.concept_table)[0]
        var_temp['unittype'] = list(x.unittype)[0]
        var_temp['nConceptPatients'] = nConceptPatients
        var_temp['nTotalPatients'] = nTotalPatients
        var_temp['fMissingRate'] = fMissingRate
        temp_df = temp_df.append(var_temp, ignore_index=True)
        return temp_df

    nTotalPatients = len(df.patientunitstayid.unique())
    features_df = df.groupby('concept_name').apply(lambda x : filter_with_missing_rate_concept(x, nTotalPatients, threshold)).reset_index(drop=True)
    return features_df

feature_df = concat_df.groupby('unittype').apply(lambda x : filter_with_missing_rate(x, threshold=0.05)).reset_index(drop=True)


In [ ]:
feature_df = feature_df.loc[feature_df['unittype'].isin(['MICU', 'SICU'])]
print( len(feature_df.loc[feature_df['unittype']=='MICU']), len(feature_df.loc[feature_df['unittype']=='SICU']) )
feature_df2 = pd.pivot_table(data=feature_df, index=['concept_name', 'concept_table'], columns='unittype', values='fMissingRate', fill_value=0).reset_index()
feature_df2['gap'] = abs(feature_df2['MICU'] - feature_df2['SICU'])
feature_df2.sort_values('gap')
feature_df2.to_csv('feature_df2.csv')

In [ ]:
filter = {'med': 0.05, 'adm':0.05, 'lab':0.7}

index_list = []
def filterwithmissingrate(df, unittype, filter):
    unit_df = pd.DataFrame()
    for tName in filter:
        print('concept_table == \'{}\' and {} > {}'.format(tName, unittype, filter[tName]))
        feature_table_df = df.query('concept_table == \'{}\' and {} > {}'.format(tName, unittype, filter[tName]))
        if not feature_table_df.empty:
            unit_df = pd.concat([unit_df, feature_table_df], axis=0)
    return unit_df
   
MICU_feature = filterwithmissingrate(feature_df2, 'MICU', filter) 
SICU_feature = filterwithmissingrate(feature_df2, 'SICU', filter)


In [ ]:
len(SICU_feature.concept_name.unique()), len(MICU_feature.concept_name.unique())

In [ ]:
concat_df[concat_df['concept_table']=='lab']

In [ ]:
SICU_sub_df = concat_df.loc[(concat_df['unittype']=='SICU') & (concat_df['concept_name'].isin(SICU_feature.concept_name.unique()))]
SICU_sub_df = pd.pivot_table(data=SICU_sub_df, index=['patientunitstayid', 'unittype'], columns='concept_name', values='concept_value').reset_index()
SICU_sub_df

In [ ]:
MICU_sub_df = concat_df.loc[(concat_df['unittype']=='MICU') & (concat_df['concept_name'].isin(MICU_feature.concept_name.unique()))]
MICU_sub_df = pd.pivot_table(data=MICU_sub_df, index=['patientunitstayid', 'unittype'], columns='concept_name', values='concept_value').reset_index()
MICU_sub_df

In [ ]:
cols = list(MICU_sub_df.columns)
cols

In [ ]:
SICU_sub_df.columns

In [ ]:
def getDuplicatedColumns(df1, df2):
    dup_col_list = []
    for col1 in df1.columns:
        for col2 in df2.columns:
            if col1.lower() == col2.lower() and col1 != 'patientunitstayid':
                dup_col_list.append(col2)
    return dup_col_list
                
dup_col_list = getDuplicatedColumns(SICU_main_df, SICU_sub_df)

In [ ]:
# main : hosptial + apache
# sub : lab, admission_drug, med
SICU_main_df = patient_apc_df.loc[patient_apc_df['unittype']=='SICU']
MICU_main_df = patient_apc_df.loc[patient_apc_df['unittype']=='MICU']

MICU_sub_df.drop(columns=getDuplicatedColumns(MICU_main_df, MICU_sub_df), axis=1, inplace=True)
SICU_sub_df.drop(columns=getDuplicatedColumns(SICU_main_df, SICU_sub_df), axis=1, inplace=True)

SICU_df = pd.merge(SICU_main_df, SICU_sub_df, left_on='patientunitstayid', right_on='patientunitstayid', how='inner')
MICU_df = pd.merge(MICU_main_df, MICU_sub_df, left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [ ]:
len(SICU_df.columns), len(MICU_df.columns)

In [ ]:
SICU_df.to_feather('{}/SICU_df.feather'.format(data_dir))
MICU_df.to_feather('{}/MICU_df.feather'.format(data_dir))